In [ ]:
import copy
import logging

import numpy as np

from atmodeller import (
    InteriorAtmosphere,
    Planet,
    Species,
    SpeciesCollection,
    bulk_silicate_earth_abundances,
    debug_logger,
    earth_oceans_to_hydrogen_mass,
)
from atmodeller.solubility import get_solubility_models
from atmodeller.thermodata import IronWustiteBuffer

logger = debug_logger()
logger.setLevel(logging.INFO)
# For more output use DEBUG
logger.setLevel(logging.DEBUG)

# Trappist 1-e models from Bower et al. (2025)

Parameters for the simulations

In [ ]:
# In the paper we perform 10,000 simulations
# number_of_realisations = 10000
number_of_realisations = 100
magma_ocean_temperature = 1800
# In the paper we perform simulations at 0.1 and 1.0 melt fraction
mantle_melt_fraction = 1.0

# Venus-like surface temperature
hot_surface_temperature = 740

# Corresponds to the highest temperature at which all condensates can form
medium_surface_temperature = 380

# Corresponds to the equilibrium temperature of Trappist-1e
cold_surface_temperature = 280

# For no particular reason, use 24 as the random seed
RANDOM_SEED = 24
WRITE_OUTPUT = True

# These are just to know at which temperatures condensates are stable
# Temperature must be equal to or less than these values for water condensation and sulfur
# freezing to be applicable.
# water_condensation = 600.0  # First thermo data for H2O(l)
# sulfur_freezing = 380.0  # 388.36  # First thermo data for S(cr)

# For naming output data
magma_ocean_temp_str: str = f"{magma_ocean_temperature:0.0f}"
hot_temp_str: str = f"{hot_surface_temperature:0.0f}"
medium_temp_str: str = f"{medium_surface_temperature:0.0f}"
cold_temp_str: str = f"{cold_surface_temperature:0.0f}"

## High temperature atmospheric diversity (gas + C_cr)

Parameter space to probe. Generate once to ensure the same values are used across the high temperature models allowing direct comparison.

In [ ]:
np.random.seed(RANDOM_SEED)
log10_number_oceans = np.random.uniform(-1, 1, number_of_realisations)
log10_ch_ratios = np.random.uniform(-1, 1, number_of_realisations)
fO2_log10_shifts = np.random.uniform(-5, 5, number_of_realisations)

h_kg = earth_oceans_to_hydrogen_mass(10**log10_number_oceans)
c_kg = h_kg * 10**log10_ch_ratios

trappist1e_mantle_mass = 2.912e24
trappist1e_planet_mass = trappist1e_mantle_mass / (1 - 0.295334691460966)
trappist1e_surface_radius = 5.861e6

fugacity_constraints = {"O2_g": IronWustiteBuffer(fO2_log10_shifts)}

Species that we need

In [ ]:
H2O_g = Species.create_gas("H2O")
H2_g = Species.create_gas("H2")
O2_g = Species.create_gas("O2")
CO_g = Species.create_gas("CO")
CO2_g = Species.create_gas("CO2")
CH4_g = Species.create_gas("CH4")
N2_g = Species.create_gas("N2")
H3N_g = Species.create_gas("H3N")
S2_g = Species.create_gas("S2")
H2S_g = Species.create_gas("H2S")
O2S_g = Species.create_gas("O2S")
OS_g = Species.create_gas("OS")
Cl2_g = Species.create_gas("Cl2")
ClH_g = Species.create_gas("ClH")
# Graphite can also be present in high temperature atmospheres
C_cr = Species.create_condensed("C")
# Condensates below are used for cooler atmospheres
H2O_l = Species.create_condensed("H2O", state="l")
S_cr = Species.create_condensed("S")
ClH4N_cr = Species.create_condensed("ClH4N")

Species to consider, where solubility is not included

In [ ]:
species_nosol = SpeciesCollection(
    (
        H2_g,
        H2O_g,
        O2_g,
        CO_g,
        CO2_g,
        CH4_g,
        N2_g,
        H3N_g,
        S2_g,
        H2S_g,
        O2S_g,
        OS_g,
        Cl2_g,
        ClH_g,
        C_cr,
    )
)

TRAPPIST-1e planet properties

Mass and radius measurements from Agol et al. 2021; Mantle mass determined assuming same proportion as Earth

In [ ]:
trappist1e_magma_ocean = Planet(
    surface_temperature=magma_ocean_temperature,
    planet_mass=trappist1e_planet_mass,
    surface_radius=trappist1e_surface_radius,
    mantle_melt_fraction=mantle_melt_fraction,
)

Earth planet properties, which are required to scale the bulk volatile inventories for Trappist-1e. Default parameters are Earth so we only need to specify the temperature.

In [ ]:
earth = Planet(surface_temperature=magma_ocean_temperature)

In [ ]:
earth_bse = bulk_silicate_earth_abundances()

Compute the reservoir sizes for TRAPPIST 1-e, assuming the same ppmw as Earth:

In [ ]:
trappist1e_bse = copy.deepcopy(earth_bse)
mass_scale_factor = trappist1e_magma_ocean.mantle_mass / earth.mantle_mass

for element, values in trappist1e_bse.items():
    trappist1e_bse[element] = {key: value * mass_scale_factor for key, value in values.items()}  # type: ignore

trappist1e_bse

In [ ]:
interior_atmosphere_nosol = InteriorAtmosphere(species_nosol)

mass_constraints = {
    "H": h_kg,
    "C": c_kg,
    "N": trappist1e_bse["N"]["mean"],
    "S": trappist1e_bse["S"]["mean"],
    "Cl": trappist1e_bse["Cl"]["mean"],
}

interior_atmosphere_nosol.solve(
    planet=trappist1e_magma_ocean,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
)
output_nosol = interior_atmosphere_nosol.output

# Quick look at the solution
# solution_nosol = output_nosol.quick_look()

# Get complete solution as a dictionary
# solution_nosol asdict = output_nosol.asdict()
# logger.info(solution_nosol_asdict)

if WRITE_OUTPUT:
    # Write the complete solution to Excel
    output_nosol.to_excel(f"t1e_{magma_ocean_temp_str}K_no_solubility")

    # Write the data to a pickle file with dataframes
    output_nosol.to_pickle(f"t1e_{magma_ocean_temp_str}K_no_solubility")

Cases with solubility

In [ ]:
solubility_models = get_solubility_models()

In [ ]:
H2O_g = Species.create_gas("H2O", solubility=solubility_models["H2O_basalt_dixon95"])
H2_g = Species.create_gas("H2", solubility=solubility_models["H2_basalt_hirschmann12"])
O2_g = Species.create_gas("O2")
CO_g = Species.create_gas("CO", solubility=solubility_models["CO_basalt_yoshioka19"])
CO2_g = Species.create_gas("CO2", solubility=solubility_models["CO2_basalt_dixon95"])
CH4_g = Species.create_gas("CH4", solubility=solubility_models["CH4_basalt_ardia13"])
N2_g = Species.create_gas("N2", solubility=solubility_models["N2_basalt_libourel03"])
H3N_g = Species.create_gas("H3N")
S2_g = Species.create_gas("S2", solubility=solubility_models["S2_basalt_boulliung23"])
H2S_g = Species.create_gas("H2S")
O2S_g = Species.create_gas("O2S")
OS_g = Species.create_gas("OS")
Cl2_g = Species.create_gas("Cl2", solubility=solubility_models["Cl2_basalt_thomas21"])
ClH_g = Species.create_gas("ClH")
C_cr = Species.create_condensed("C")

species_withsol = SpeciesCollection(
    (
        H2_g,
        H2O_g,
        O2_g,
        CO_g,
        CO2_g,
        CH4_g,
        N2_g,
        H3N_g,
        S2_g,
        H2S_g,
        O2S_g,
        OS_g,
        Cl2_g,
        ClH_g,
        C_cr,
    )
)

In [ ]:
interior_atmosphere_withsol = InteriorAtmosphere(species_withsol)

mass_constraints = {
    "H": h_kg,
    "C": c_kg,
    "N": trappist1e_bse["N"]["mean"],
    "S": trappist1e_bse["S"]["mean"],
    "Cl": trappist1e_bse["Cl"]["mean"],
}

interior_atmosphere_withsol.solve(
    planet=trappist1e_magma_ocean,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
)
output_withsol = interior_atmosphere_withsol.output

# Quick look at the solution
# solution = output_withsol.quick_look()

# Get complete solution as a dictionary
# solution_withsol_asdict = output_withsol.asdict()
# logger.info(solution_withsol_asdict)

if WRITE_OUTPUT:
    # Write the complete solution to Excel
    output_withsol.to_excel(f"t1e_{magma_ocean_temp_str}K_with_solubility")

    # Write the data to a pickle file with dataframes
    output_withsol.to_pickle(f"t1e_{magma_ocean_temp_str}K_with_solubility")

# Solidified planet

Get the starting abundances for the calculations:

In [ ]:
withsol_dict = output_withsol.asdict()

# Create a dictionary with the series
mass_constraints = {
    "H": withsol_dict["element_H"]["atmosphere_mass"],
    "S": withsol_dict["element_S"]["atmosphere_mass"],
    "N": withsol_dict["element_N"]["atmosphere_mass"],
    "O": withsol_dict["element_O"]["atmosphere_mass"],
    # For C, we need to add the condensed mass to the atmosphere mass since the assumption is that
    # they remain in equilibrium
    "C": withsol_dict["element_C"]["atmosphere_mass"]
    + withsol_dict["element_C"]["condensed_mass"],
    "Cl": withsol_dict["element_Cl"]["atmosphere_mass"],
}

# Hot Venus-like surface temperature

In [ ]:
interior_atmosphere_hot = InteriorAtmosphere(species_nosol)

trappist1e_hot = Planet(
    surface_temperature=hot_surface_temperature,
    planet_mass=trappist1e_planet_mass,
    surface_radius=trappist1e_surface_radius,
    mantle_melt_fraction=0.0,  # Always zero because the planet is solidified
)

interior_atmosphere_hot.solve(
    planet=trappist1e_hot,
    mass_constraints=mass_constraints,
    initial_log_number_density=interior_atmosphere_withsol.output.log_number_density,
    initial_log_stability=interior_atmosphere_withsol.output.log_stability,
)
output_hot = interior_atmosphere_hot.output

if WRITE_OUTPUT:
    # Write the complete solution to Excel
    output_hot.to_excel(f"t1e_{hot_temp_str}K")

    # Write the data to a pickle file with dataframes
    output_hot.to_pickle(f"t1e_{hot_temp_str}K")

# Medium surface temperature where all condensates can be stable

In [ ]:
species_with_all_condensates = SpeciesCollection(
    (
        H2_g,
        H2O_g,
        O2_g,
        CO_g,
        CO2_g,
        CH4_g,
        N2_g,
        H3N_g,
        S2_g,
        H2S_g,
        O2S_g,
        OS_g,
        Cl2_g,
        ClH_g,
        C_cr,
        H2O_l,
        S_cr,
        ClH4N_cr,
    )
)

In [ ]:
interior_atmosphere_medium = InteriorAtmosphere(species_with_all_condensates)

trappist1e_medium = Planet(
    surface_temperature=medium_surface_temperature,
    planet_mass=trappist1e_planet_mass,
    surface_radius=trappist1e_surface_radius,
    mantle_melt_fraction=0.0,  # Always zero because the planet is solidified
)

interior_atmosphere_medium.solve(
    planet=trappist1e_medium,
    mass_constraints=mass_constraints,
)
output_medium = interior_atmosphere_medium.output

if WRITE_OUTPUT:
    # Write the complete solution to Excel
    output_medium.to_excel(f"t1e_{medium_temp_str}K")

    # Write the data to a pickle file with dataframes
    output_medium.to_pickle(f"t1e_{medium_temp_str}K")

# Cold surface temperature where all condensates can be stable

In [ ]:
interior_atmosphere_cold = InteriorAtmosphere(species_with_all_condensates)

trappist1e_cold = Planet(
    surface_temperature=cold_surface_temperature,
    planet_mass=trappist1e_planet_mass,
    surface_radius=trappist1e_surface_radius,
    mantle_melt_fraction=0.0,  # Always zero because the planet is solidified
)

interior_atmosphere_cold.solve(
    planet=trappist1e_cold,
    mass_constraints=mass_constraints,
    initial_log_number_density=interior_atmosphere_medium.output.log_number_density,
    initial_log_stability=interior_atmosphere_medium.output.log_stability,
)
output_cold = interior_atmosphere_cold.output

if WRITE_OUTPUT:
    # Write the complete solution to Excel
    output_cold.to_excel(f"t1e_{cold_temp_str}K")

    # Write the data to a pickle file with dataframes
    output_cold.to_pickle(f"t1e_{cold_temp_str}K")